In [1]:
# !conda update scikit-learn --yes
# !conda update sdv --yes

In [2]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')

import PETs_Experiment

In [3]:
loader = PETs_Experiment.Loader(filepath = '../[sunset]/data/[Adt Income] adult.csv'
                               ,na_values = {k : '?' for k in ['workclass'
                                                              ,'occupation'
                                                              ,'native-country'
                                                              ]
                                            }
                               )
print(loader.data.head(1))

   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [4]:
splitter = PETs_Experiment.Splitter(data              = loader.data
                                   ,num_samples       = 30
                                   ,train_split_ratio = 0.8)
print(splitter.data[1]['train'     ].shape[0])
print(splitter.data[1]['validation'].shape[0])
print(splitter.data[1]['train'     ].head(1) )
print(splitter.data[1]['validation'].head(1) )

39073
9769
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
   age workclass  fnlwgt education  educational-num      marital-status  \
0   38   Private   89814   HS-grad                9  Married-civ-spouse   

        occupation relationship   race gender  capital-gain  capital-loss  \
0  Farming-fishing      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              50  United-States  <=50K  


In [5]:
preproc = PETs_Experiment.Preprocessor(
     data = splitter.data[1]['train']
    ,missing=True
        ,missing_method  = 'drop'
        ,missing_columns = None
    ,outlier=True
        ,outlier_method  = 'IQR'
        ,outlier_columns = {'ignore': ['hours-per-week']}
    ,encoding=True
        ,encoding_method  = 'Label'
        ,encoding_columns = None
    ,scaling=True
        ,scaling_method  = 'Standard'
        ,scaling_columns = {'focus' : 'fnlwgt'}
)
print('\n\n\n')
print(preproc.data.head(1))
print('\n\n\n')
print(preproc.encoder)
print('\n\n\n')
print(preproc.scaler)

Preprocessor - Missingist (Drop): Dropped 2879 rows.
Preprocessor - Outlierist (IQR): Dropped  1074 rows on fnlwgt         . Kept [-62753.0, 417879.0] only.
Preprocessor - Outlierist (IQR): Dropped  1748 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   228 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped   217 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  2989 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6062 in 36194 rows.
Preprocessor - Encoder (Label): Column education       been labelized from 0 to 13.
Preprocessor - Encoder (Label): Column income          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column race            been labelized from 0 to  4.
Preprocessor - Encoder (Label): Column gender          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column native-country  bee

In [6]:
synthesizer = PETs_Experiment.Synthesizer(data = preproc.data
                                         ,synthesizing_method = 'sdv-singletable-gaussiancoupula'
                                         )
synthesizer.fit_sample()
print(synthesizer.data_syn.head(1))

Synthesizer (SDV - SingleTable): Metafile loading time: 0.0281 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.


c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 15.1579 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30132 rows (same as raw) in 1.6513 sec.
   age  workclass    fnlwgt  education  educational-num  marital-status  \
0   43          2  1.867811         10                8               0   

   occupation  relationship  race  gender  capital-gain  capital-loss  \
0           7             4     4       1             0             0   

   hours-per-week  native-country  income  
0              49              39       0  


In [7]:
postproc = PETs_Experiment.Postprocessor(data = synthesizer.data_syn
                                        ,encoder = getattr(preproc ,'encoder' ,None)
                                        ,scaler  = getattr(preproc ,'scaler'  ,None)
                                        )
print(postproc.data.head(1))

Postprocessor - Scaler (StandardScaler): Decoding fnlwgt.
Postprocessor - Encoder (LabelEncoder): Decoding education.
Postprocessor - Encoder (LabelEncoder): Decoding income.
Postprocessor - Encoder (LabelEncoder): Decoding race.
Postprocessor - Encoder (LabelEncoder): Decoding gender.
Postprocessor - Encoder (LabelEncoder): Decoding native-country.
Postprocessor - Encoder (LabelEncoder): Decoding occupation.
Postprocessor - Encoder (LabelEncoder): Decoding relationship.
Postprocessor - Encoder (LabelEncoder): Decoding workclass.
Postprocessor - Encoder (LabelEncoder): Decoding marital-status.
   age workclass         fnlwgt education  educational-num marital-status  \
0   43   Private  343391.443081   HS-grad                8       Divorced   

      occupation relationship   race gender  capital-gain  capital-loss  \
0  Other-service    Unmarried  White   Male             0             0   

   hours-per-week native-country income  
0              49     Yugoslavia  <=50K  


In [8]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
import PETs_Experiment

para_Executor = {
     'Loader_filepath'  : '../[sunset]/data/[Adt Income] adult.csv'
    ,'Loader_na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
    ,'Splitter_num_samples'       : 2
    ,'Splitter_train_split_ratio' : 0.8
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    ,'Preprocessor_scaling'         : True
    ,'Preprocessor_scaling_method'  : 'Standard'
    ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer_synthesizing_method' : 'sdv-singletable-gaussiancoupula'
}

executor = PETs_Experiment.Executor(**para_Executor)
executor.run()


for k ,v in executor.loader.items():
    print(k)
    print(v.data.head(1))

====== ====== ====== ====== ====== ======
Executor: Total execution time: 90.1305 sec.
====== ====== ====== ====== ====== ======
[Adt Income] adult.csv
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [9]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
import PETs_Experiment

para_Executor = {
     'Loader' : {
         'adult'  : {'filepath'   : '../[sunset]/data/[Adt Income] adult.csv'
                    ,'na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
        }
        ,'NHANES' : {'filepath'      : '../[sunset]/data/[NHANES] B.csv'
                    ,'header_exist' : False
                    ,'header_names' : ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
        }
     }
    ,'Splitter' : {
         '0.8'  : {'num_samples'       : 2
                  ,'train_split_ratio' : 0.8
        }
        ,'0.05' : {'num_samples'       : 1
                  ,'train_split_ratio' : 0.05
        }
    }
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    # ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    # ,'Preprocessor_scaling'         : True
    # ,'Preprocessor_scaling_method'  : 'Standard'
    # ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer': {
        'GaussianCoupula' : {'synthesizing_method' : 'sdv-singletable-gaussiancoupula'}
       ,'CoupulaGAN'      : {'synthesizing_method' : 'sdv-singletable-coupulagan'}
       ,'CTGAN'           : {'synthesizing_method' : 'sdv-singletable-ctgan'}
       ,'TVAE'            : {'synthesizing_method' : 'sdv-singletable-tvae'}
    }
}

executor = PETs_Experiment.Executor(**para_Executor)
executor.run()
    

====== ====== ====== ====== ====== ======
Executor: Total execution time: 145.0886 sec.
====== ====== ====== ====== ====== ======


In [28]:




from PETs_Experiment import Loader
self = executor    


def test(key ,para):
    print(key)
    return key , 'para'




with ThreadPoolExecutor() as async_executor:
    futures = [async_executor.submit(test, _key, _para)
                for _key ,_para in self.para['Loader_setting'].items()]

    for future in futures:
        key, loader_result = future.result()
        self.loader[key] = loader_result

self.loader

adult
NHANES


{'adult': 'para', 'NHANES': 'para'}